# What is Selenium?

## 1

What environment do we use?

See [What is Docker?](What%20is%20Docker.ipynb)

Run Selenium in Docker?


In [1]:
var importer = require('../Core');var fs = require('fs');// add some run commands to the bash scriptvar bashToRun = (code) => code.split('\n').filter(n => n.trim().length > 0).map(l => 'RUN ' + l.trim())    .join('\n').replace(/\\\s*\nRUN\s*/ig, '\\\n ');// create a selenium Dockerfile with a vnc connectionvar seleniumDocker = (outputFile) => {    return importer.interpret([        'run selenium',        // add some extra services        'linux dev tools',        'vnc html',        'vnc docker'    ]).then(r => {        // convert some results to Docker RUN commands instead of bash        r[1].code = bashToRun(r[1].code);        r[2].code = bashToRun(r[2].code);        return r.map(r => r.code).join('\n');    })        .then(r => {            // save the Dockerfile            fs.writeFileSync(outputFile, r);            return r;        })        .catch(e => console.log(e))};module.exports = seleniumDocker;

[Function: seleniumDocker]

How to set up selenium server?

In [2]:
var importer = require('../Core');var path = require('path');var fs = require('fs');var DOWNLOAD_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads');var PROFILE_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.defaultProfile');var execCmd;var getSeleniumServer = (name = 'act-selenium') => {    try {        fs.mkdirSync(DOWNLOAD_DIR);    } catch (err) {        if (err.code != 'EEXIST') {            throw err;        }    }    try {        fs.mkdirSync(PROFILE_DIR);    } catch (err) {        if (err.code != 'EEXIST') {            throw err;        }    }    try {        fs.unlinkSync(path.join(PROFILE_DIR, 'SingletonLock'));    } catch (err) {        if (err.code != 'ENOENT') {            throw err;        }    }    var DOCKERFILE = path.resolve(path.join(__dirname, 'Dockerfile'));    return seleniumDocker(DOCKERFILE)        .then(() => importer.import('spawn child process'))        .then(execCmd => execCmd('docker ps -a'))        .then(r => {            if (r[0].indexOf(name) > -1) {                return execCmd('docker stop ' + name)                    .then(r => new Promise(resolve =>                        setTimeout(() => resolve(r), 1000)))                    .then(() => execCmd('docker rm ' + name));            }        })        .then(() => new Promise(resolve =>            setTimeout(() => resolve(), 1000)))        .then(() => {            var build = 'docker build -t ' + name + ' "'                + path.dirname(DOCKERFILE) + '"\n'                + 'docker run --shm-size=3g -d '                + '--name ' + name + ' '                + '-p 8888:8888 '                + '-p 6080:6080 '                + '-p 5900:5900 '                + '-p 4444:4444 '                + '-p 4200:4200 '                + '-p 3000:3000 '                // TODO: add profile dir back in when permissions works on windows                + '-v "' + DOWNLOAD_DIR + '":/data/downloads '                + name + '\n';            return execCmd(build)        })        .then(r => new Promise(resolve => setTimeout(() => resolve(r), 6000)))        .catch(e => console.log(e))};module.exports = getSeleniumServer;getSeleniumServer;

[Function: getSeleniumServer]

In [4]:
$$.async();getSeleniumServer()    .then(r => $$.sendResult(r))    .catch(e => $$.sendError(e));

ReferenceError: getSeleniumServer is not defined

# 2

How to connect to VNC through the web browser?

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true&reconnect=true) in a separate browser


In [1]:
var urlEncode = (obj) => {    return Object.keys(obj).map(k => encodeURIComponent(k) + '=' + encodeURIComponent(obj[k])).join('&');}var vncIframe = (options = {    password: 'secret',    host: 'localhost',    port: 6080,    autoconnect: true,    resize: 'downscale',    view_only: true,    reconnect: true}) => `<div style="display:block; width:100%; padding-bottom:70%;position:relative;"><iframe id="vnc"style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;" src="//local:6080/vnc.html?${urlEncode(options)}"></iframe></div>`;module.exports = vncIframe;vncIframe;

[Function: vncIframe]

In [2]:
$$.mime({'text/html': vncIframe()});

## 3

How to run a selenium cell on the Docker machine?



In [1]:
var importer = require('../Core');var runSeleniumCell = (search) => {    var client, getCredentials, getAllXPath, getAllUntil;    return importer.import([        'webdriver client',        'decrypt password'    ])        .then(r => {            getCredentials = r[1];            client = r[0]();            return importer.import([                'all xpath elements',                'get all elements until'            ], {client});        })        .then(r => {            getAllXPath = r[0];            getAllUntil = r[1];            return importer.import(search, {                browser: client,                client,                getCredentials,                getAllXPath,                getAllUntil            });        })        .catch(e => console.log(e))};module.exports = runSeleniumCell;

[Function: runSeleniumCell]

In [2]:
$$.async()runSeleniumCell('test docker selenium')    .then(r => $$.sendResult(r))    .catch(e => $$.sendError(e));

Initializing webdriver on localhost
{ Error
  message: '<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="utf-8">\n<title>Error</title>\n</head>\n<body>\n<pre>Error: not implemented!<br> &nbsp; &nbsp;at router.get (evalmachine.&lt;anonymous&gt;:22:15)<br> &nbsp; &nbsp;at Layer.handle [as handle_request] (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/layer.js:95:5)<br> &nbsp; &nbsp;at next (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/route.js:137:13)<br> &nbsp; &nbsp;at Route.dispatch (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/route.js:112:3)<br> &nbsp; &nbsp;at Layer.handle [as handle_request] (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/layer.js:95:5)<br> &nbsp; &nbsp;at /Users/briancullinan/jupytangular2/node_modules/express/lib/router/index.js:281:22<br> &nbsp; &nbsp;at param (/Users/briancullinan/jupytangular2/node_modules/express/lib/router/index.js:354:14)<br> &nbsp; &nbsp;at param

Test docker selenium?

In [8]:
if (typeof client !== 'undefined') {    client.url('https://purchasesprint.actops.com');    client.click('[href*="/auth"], [routerlink*="/auth"]');    module.exports = client;}


TODO:

Auto fill fields in a live browser using the 

Transfer login state to current browser?

